In [1]:
import biosteam as bst
from biosteam import settings
bst.nbtutorial() # Light-mode html diagrams and filter warnings
settings.set_thermo(['Water', 'Methanol'])
feed = bst.Stream(Water=50, Methanol=20)
feed.show()

Stream: s1
 phase: 'l', T: 298.15 K, P: 101325 Pa
 flow (kmol/hr): Water     50
                 Methanol  20


In [2]:
feed.price = 0.15 # USD/kg
feed.cost # USD/hr

231.24018

In [3]:
settings.CEPCI # Default year is 2017

567.5

In [4]:
settings.CEPCI = 603.1 # To year 2018

In [5]:
settings.electricity_price # Default price (USD/kWhr)

0.0782

In [6]:
settings.electricity_price = 0.065 # Adjust price

In [7]:
settings.electricity_price = 0.065 # Adjust price

In [8]:
settings.cooling_agents # All available cooling agents

[<UtilityAgent: cooling_water>,
 <UtilityAgent: chilled_water>,
 <UtilityAgent: chilled_brine>,
 <UtilityAgent: propane>]

In [9]:
cooling_water = settings.get_cooling_agent('cooling_water')
cooling_water.show() # A UtilityAgent

UtilityAgent: cooling_water
 heat_transfer_efficiency: 1.000
 heat_transfer_price: 0 USD/kJ
 regeneration_price: 0.000488 USD/kmol
 T_limit: 325 K
 phase: 'l'
 T: 305.37 K
 P: 101325 Pa
 flow (kmol/hr): Water  1


In [10]:
# Price of regenerating the utility in USD/kmol
cooling_water.regeneration_price

0.00048785

In [11]:
# Other utilities may be priced for amount of heat transfered in USD/kJ
chilled_water = settings.get_cooling_agent('chilled_water')
chilled_water.heat_transfer_price

5e-06

In [12]:
cooling_water.T = 302 # Change the temperature of cooling water (K)

In [13]:
settings.heating_agents # All available heating agents

[<UtilityAgent: low_pressure_steam>,
 <UtilityAgent: medium_pressure_steam>,
 <UtilityAgent: high_pressure_steam>]

In [14]:
lps = settings.get_heating_agent('low_pressure_steam') # A UtilityAgent
lps.show() # Note that because utility changes phase, T_limit is None

UtilityAgent: low_pressure_steam
 heat_transfer_efficiency: 0.950
 heat_transfer_price: 0 USD/kJ
 regeneration_price: 0.238 USD/kmol
 T_limit: None
 phase: 'g'
 T: 412.19 K
 P: 344738 Pa
 flow (kmol/hr): Water  1


In [15]:
lps.regeneration_price = 0.20 # Adjust price (USD/kmol)

In [16]:
from biosteam import units

# Specify vapor fraction and isobaric conditions
F1 = units.Flash('F1', V=0.1, P=101325)
F1.show()

Flash: F1
ins...
[0] missing stream
outs...
[0] s2
    phase: 'l', T: 298.15 K, P: 101325 Pa
    flow: 0
[1] s3
    phase: 'l', T: 298.15 K, P: 101325 Pa
    flow: 0


In [17]:
F1.ins

[<MissingStream>]

In [18]:
F1.outs

[<Stream: s2>, <Stream: s3>]

In [19]:
F1.ins[0] = feed
F1.show()

Flash: F1
ins...
[0] s1
    phase: 'l', T: 298.15 K, P: 101325 Pa
    flow (kmol/hr): Water     50
                    Methanol  20
outs...
[0] s2
    phase: 'l', T: 298.15 K, P: 101325 Pa
    flow: 0
[1] s3
    phase: 'l', T: 298.15 K, P: 101325 Pa
    flow: 0


In [20]:
F1.simulate()
F1.show()

Flash: F1
ins...
[0] s1
    phase: 'l', T: 298.15 K, P: 101325 Pa
    flow (kmol/hr): Water     50
                    Methanol  20
outs...
[0] s2
    phase: 'g', T: 352.87 K, P: 101325 Pa
    flow (kmol/hr): Water     2.59
                    Methanol  4.41
[1] s3
    phase: 'l', T: 352.87 K, P: 101325 Pa
    flow (kmol/hr): Water     47.4
                    Methanol  15.6


In [21]:
print(F1.results()) # Default returns DataFrame object with units

Flash                                               Units            F1
Low pressure steam  Duty                            kJ/hr      5.91e+05
                    Flow                          kmol/hr          15.2
                    Cost                           USD/hr          3.05
Design              Vessel type                                Vertical
                    Length                             ft          44.5
                    Diameter                           ft           0.5
                    Weight                             lb           718
                    Wall thickness                     in          0.25
                    Vessel material                        Carbon steel
Purchase cost       Heat exchanger - Double pipe      USD       4.3e+03
                    Vertical pressure vessel          USD       1.2e+04
                    Platform and ladders              USD      3.82e+03
Total purchase cost                                   USD      2

In [22]:
F1.net_duty # Duty [kJ / hr]

591499.1453087364

In [23]:
F1.net_power # Electricity consumption [kW]

0.0

In [24]:
[F1.feed, F1.vapor, F1.liquid] # Inlet feed and vapor and liquid outlets

[<Stream: s1>, <Stream: s2>, <Stream: s3>]

In [25]:
M1 = units.Mixer('M1')
S1 = units.Splitter('S1', outs=('liquid_recycle', 'liquid_product'),
                    split=0.5) # Split to 0th output stream
F1.outs[0].ID = 'vapor_product'
F1.outs[1].ID = 'liquid'

In [26]:
bst.main_flowsheet.diagram()
# Note that empty streams are dashed and the 
# width of streams depend on their flow rates (by mass)

In [28]:
feed = bst.Stream('feed', Methanol=100, Water=450)

# Broken down -pipe- notation
[S1-0, feed]-M1     # M1.ins[:] = [S1.outs[0], feed]
M1-F1               # F1.ins[:] = M1.outs
F1-1-S1             # S1.ins[:] = [F1.outs[1]]

# All together
[S1-0, feed]-M1-F1-1-S1;

In [29]:
bst.main_flowsheet.diagram() # bst.F.diagram() also works

In [30]:
flowsheet_sys = bst.main_flowsheet.create_system('flowsheet_sys')
flowsheet_sys.show()

System: flowsheet_sys
ins...
[0] feed
    phase: 'l', T: 298.15 K, P: 101325 Pa
    flow (kmol/hr): Water     450
                    Methanol  100
outs...
[0] vapor_product
    phase: 'g', T: 352.87 K, P: 101325 Pa
    flow (kmol/hr): Water     2.59
                    Methanol  4.41
[1] liquid_product
    phase: 'l', T: 298.15 K, P: 101325 Pa
    flow: 0


In [31]:
sys = bst.System('sys', path=(M1, F1, S1), recycle=S1-0) # recycle=S1.outs[0]
sys.show()

System: sys
ins...
[0] feed
    phase: 'l', T: 298.15 K, P: 101325 Pa
    flow (kmol/hr): Water     450
                    Methanol  100
outs...
[0] vapor_product
    phase: 'g', T: 352.87 K, P: 101325 Pa
    flow (kmol/hr): Water     2.59
                    Methanol  4.41
[1] liquid_product
    phase: 'l', T: 298.15 K, P: 101325 Pa
    flow: 0


In [32]:
sys.simulate()
sys.show()

System: sys
Highest convergence error among components in recycle
stream S1-0 after 4 loops:
- flow rate   3.12e-01 kmol/hr (0.56%)
- temperature 3.55e-02 K (0.0099%)
ins...
[0] feed
    phase: 'l', T: 298.15 K, P: 101325 Pa
    flow (kmol/hr): Water     450
                    Methanol  100
outs...
[0] vapor_product
    phase: 'g', T: 359.27 K, P: 101325 Pa
    flow (kmol/hr): Water     53.5
                    Methanol  46.6
[1] liquid_product
    phase: 'l', T: 359.27 K, P: 101325 Pa
    flow (kmol/hr): Water     397
                    Methanol  53.7


In [33]:
print(F1.results())

Flash                                                 Units            F1
Low pressure steam  Duty                              kJ/hr      6.76e+06
                    Flow                            kmol/hr           174
                    Cost                             USD/hr          34.8
Design              Vessel type                                  Vertical
                    Length                               ft          43.5
                    Diameter                             ft           2.5
                    Weight                               lb       3.6e+03
                    Wall thickness                       in          0.25
                    Vessel material                          Carbon steel
Purchase cost       Heat exchanger - Double pipe        USD       4.3e+03
                    Heat exchanger - Floating head      USD      2.29e+04
                    Vertical pressure vessel            USD      2.79e+04
                    Platform and ladde

In [34]:
sys.power_utility.show()

PowerUtility:
 consumption: 0 kW
 production: 0 kW
 rate: 0 kW
 cost: 0 USD/hr


In [35]:
for i in sys.heat_utilities: i.show()

HeatUtility: low_pressure_steam
 duty: 6.76e+06 kJ/hr
 flow: 174 kmol/hr
 cost: 34.8 USD/hr
